<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Recognition_Xception_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import Xception
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test'
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1'
filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/vgg16_GDPS_xfer_weights.hdf5'
nb_epochs = 50
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1/'

SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
print("Input parameters are assigned")

Input parameters are assigned


In [4]:
# image data generation
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/'
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True,class_mode='categorical')

Found 672 images belonging to 64 classes.
Found 224 images belonging to 64 classes.


In [13]:
#Load the pretrained Network
xception_model = Xception( 
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_height,img_width,3),
    pooling=None,
    classes=1000
    
)
print("pretrained Network is loaded")

83689472/83683744 [==============================] - 6s 0us/step
pretrained Network is loaded


In [14]:
xception_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [15]:
# Freeze the layers
for layer in xception_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [18]:
model = Sequential()
model.add(xception_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(64, activation='softmax', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [0]:
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [21]:
history = model.fit_generator( train_generator, callbacks = callbacks_list, nb_epoch=nb_epochs, validation_data=validation_generator)
print('Training Completed!')

Epoch 1/50
21/21 [==============================] - 617s 29s/step - loss: 4.7985 - accuracy: 0.0476 - val_loss: 4.3905 - val_accuracy: 0.0402
Epoch 2/50
21/21 [==============================] - 13s 628ms/step - loss: 3.3543 - accuracy: 0.1949 - val_loss: 4.3425 - val_accuracy: 0.0759
Epoch 3/50
21/21 [==============================] - 14s 686ms/step - loss: 2.6814 - accuracy: 0.3199 - val_loss: 3.9829 - val_accuracy: 0.1071
Epoch 4/50
21/21 [==============================] - 14s 689ms/step - loss: 1.8463 - accuracy: 0.5312 - val_loss: 4.1817 - val_accuracy: 0.0804
Epoch 5/50
21/21 [==============================] - 14s 687ms/step - loss: 1.3048 - accuracy: 0.6443 - val_loss: 5.7418 - val_accuracy: 0.0714
Epoch 6/50
21/21 [==============================] - 14s 685ms/step - loss: 0.9569 - accuracy: 0.7545 - val_loss: 6.9373 - val_accuracy: 0.1027
Epoch 7/50
21/21 [==============================] - 14s 689ms/step - loss: 0.9397 - accuracy: 0.7604 - val_loss: 5.7612 - val_accuracy: 0.0893


In [0]:
import os
def get_images(fish):
    """Load files from train folder"""
    fish_dir = train_data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

In [24]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 108kB/s 
ERROR: umap-learn 0.4.2 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
def read_image(src):
    import os
    from scipy import misc
    filepath=src
    im=misc.imread(filepath)
    import scipy.misc  as mc
     
    return mc.imresize(im,(img_width, img_height))

In [28]:
files = []
y_all = []
for fish in SIGNATURE_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)
print(len(files))
print(len(y_all))


19 photos of 001
19 photos of 002
18 photos of 003
18 photos of 004
18 photos of 006
18 photos of 009
18 photos of 012
9 photos of 013
18 photos of 014
18 photos of 015
18 photos of 016
9 photos of 017
9 photos of 018
9 photos of 019
9 photos of 020
9 photos of 021
9 photos of 022
9 photos of 023
9 photos of 024
9 photos of 025
9 photos of 026
9 photos of 027
9 photos of 028
9 photos of 029
9 photos of 030
9 photos of 031
9 photos of 032
9 photos of 033
9 photos of 034
9 photos of 035
9 photos of 036
9 photos of 037
9 photos of 038
9 photos of 039
9 photos of 040
9 photos of 041
9 photos of 042
9 photos of 043
9 photos of 044
9 photos of 045
9 photos of 046
9 photos of 047
6 photos of 048
9 photos of 049
9 photos of 050
9 photos of 051
9 photos of 052
9 photos of 053
9 photos of 054
9 photos of 055
9 photos of 056
9 photos of 057
9 photos of 058
9 photos of 059
9 photos of 060
9 photos of 061
9 photos of 062
9 photos of 063
9 photos of 064
9 photos of 065
9 photos of 066
12 photos of 0

In [31]:
X_all = np.ndarray((len(files),img_width, img_height , 3), dtype=np.uint8)
print(X_all.shape)
for i, im in enumerate(files): 
    X_all[i] = read_image(train_data_dir+im)
    print(X_all[i])   
   
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))


print(X_all.shape)

Streaming output truncated to the last 5000 lines.
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 252]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]]
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 

In [32]:
import os
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), img_width, img_height, 3), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

submission.to_csv('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureResults.csv',index=False)

12/12 [==============================] - 2s 148ms/step


In [22]:
# save model and architecture to single file
model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_xceptionfolder_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12845184  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 33,719,080
Trainable params: 12,857,600
Non-trainable params: 20,861,480
_________________________________________________________________
Saved model to disk


In [33]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_xceptionfolder_model.h5')
print("Model is Loaded")
model.summary()

Model is Loaded
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12845184  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 33,719,080
Trainable params: 12,857,600
Non-trainable params: 20,861,480
_________________________________________________________________


In [34]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click to know Name " )
output = widgets.Output()

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16_model.h5')
img_path = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/11_052.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features)
label_index=block4_pool_features.argmax()
print(label_index)
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Signature is of  "  +SIGNATURE_CLASSES[label_index])

button.on_click(on_button_clicked)
display(button, output)

[[1.07392983e-10 2.33687073e-15 1.57053474e-16 3.63361067e-07
  2.06279568e-12 5.14662590e-10 5.26927460e-11 1.13121935e-10
  1.23408850e-09 1.19379166e-07 1.47738956e-11 7.59147911e-10
  1.12327179e-08 1.42530553e-07 1.41109938e-14 2.81837327e-08
  1.50760504e-09 1.66781131e-10 1.15319581e-05 1.17537553e-07
  1.68617760e-06 4.35487846e-06 1.82786869e-10 6.03521570e-08
  2.88964270e-06 2.22059285e-07 1.76758660e-11 3.87886405e-14
  1.33578747e-11 8.05362808e-12 1.53552319e-05 4.03263585e-17
  1.60578873e-06 3.29201553e-12 6.05252168e-11 7.52721144e-06
  2.23653061e-07 2.73918385e-16 1.03516303e-13 6.31270253e-12
  2.15226503e-10 5.15620650e-06 1.58938904e-10 6.50016662e-13
  1.65134279e-05 1.07745268e-06 9.99741256e-01 3.34274441e-10
  2.03905734e-06 1.76027067e-13 4.70588737e-13 1.54899260e-09
  1.12557116e-15 3.14182067e-08 1.81715382e-04 1.67796821e-09
  8.04150323e-12 1.23990965e-12 3.76784079e-08 7.23595292e-07
  1.44528139e-11 2.06036499e-09 3.21947553e-08 5.13827945e-06]]
46


Button(description='Click to know Name ', style=ButtonStyle())

Output()

In [35]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_xceptionfolder_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
7/7 [==============================] - 5s 683ms/step
test loss, test acc: [9.064676284790039, 0.1294642835855484]


In [36]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [37]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [38]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_xceptionfolder_Intermediatemodel.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception_input (InputLayer)  (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12845184  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 33,714,920
Trainable params: 12,853,440
Non-trainable params: 20,861,480
_________________________________________________________________
Saved Intermediate model to disk


In [39]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_xceptionfolder_Intermediatemodel.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [40]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception_input (InputLayer)  (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12845184  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 33,714,920
Trainable params: 12,853,440
Non-trainable params: 20,861,480
_________________________________________________________________


In [41]:
# Training Label feature identification(y_train)
import numpy as np
batch_size=32
sample_count=672
features = np.zeros(shape=(672, 64))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(672, 64))
i = 0
for inputs_batch, labels_batch in train_generator:
  features_batch = model.predict(inputs_batch)
  features[i * batch_size: (i + 1) * batch_size] = features_batch
  labels[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels.shape)
print(features.shape)


(672, 64)
(672, 64)


In [42]:
 print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [43]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=labels
print(labels_train.shape)
print(labels)
#print(features_train)


(672, 64)
(672, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [44]:
# identification of test labels
import numpy as np
batch_size=32
sample_count=224
features_test = np.zeros(shape=(224, 64))  # Must be equal to the output of the convolutional base
labels_test = np.zeros(shape=(224,64))
i = 0
for inputs_batch, labels_batch in validation_generator:
  features_batch = model.predict(inputs_batch)
  features_test[i * batch_size: (i + 1) * batch_size] = features_batch
  labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels_test.shape)
print(features_test.shape)
print(labels_test)

(224, 64)
(224, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [45]:
# #identification of testing Labels
print(features_test.shape)
#labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)
print(features_test)

(224, 64)
(224, 64)
[[ 2.56969142  0.         13.60632896 ...  6.93547964  0.
   7.33494377]
 [ 0.          0.         15.9548521  ... 11.31710243  0.
   7.24488449]
 [ 2.08731437  0.          9.0048666  ...  0.13337316  0.
  10.65198612]
 ...
 [ 6.95205402  0.          0.         ...  0.          0.55590695
  13.64309883]
 [ 0.          0.         16.1341362  ... 14.04478359  0.
   3.27309704]
 [ 5.32686138  0.          0.98182529 ...  0.          1.13603318
   4.15720606]]


In [46]:
print(labels_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [47]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
print(X_train.shape)
print(labels_train.shape)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X,y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

(672, 64)
(672, 64)


ValueError: ignored

In [48]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [49]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 4.911
Precision: 84.615
Recall: 4.911
F-Measure: 9.283


In [50]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 30.357
Precision: 30.357
Recall: 30.357
F-Measure: 30.357


In [51]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
X, y = check_X_y(X_train, labels_train, accept_sparse=False, accept_large_sparse=True, dtype='numeric', order=None, copy=False, force_all_finite=True, ensure_2d=True, allow_nd=False, multi_output=True, ensure_min_samples=1, ensure_min_features=1, y_numeric=True, estimator=None)
# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [52]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 30.804
Precision: 75.824
Recall: 30.804
F-Measure: 43.810


In [53]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored